## **Notes**

<hr>

Create the `Next.js` app with the following command:

```bash
npx create-next-app@latest

```

Then, we set up the project folder structure as follows:

## **Folder Structure**

### **public**

The `public` folder will contain the static files, such as images and fonts.

### **fonts**

The `fonts` folder will contain custom fonts. Inside the `fonts` folder, we've a typescript file `fonts.ts` that exports the custom fonts.

```typescript
import localFont from "next/font/local";

export const satoshi = localFont({
  src: [
    {
      path: "./Satoshi-Light.woff2",
      weight: "300",
      style: "normal",
    },...
  ],
  variable: "--font-satoshi",
});
```

The above code, loads the custom font files i.e. `Satoshi-Light.woff2`, `Satoshi-Regular.woff2`, etc from the local filesystem and generates a optimized `CSS` `@font-face` rule for each font file. The `variable` property allows us to use the font as a CSS variable.

The `@font-face` rules generated by the `next/font/local` package will be automatically injected into the global CSS of the application, making the fonts available for use throughout the app as below:

```css
@font-face {
  font-family: "__satoshi";
  src: url("/_next/static/media/Satoshi-Regular.woff2") format("woff2");
  font-weight: 400;
  font-style: normal;
}
```

The `@font-face` is a CSS rule that allows us to define custom fonts that can be loaded and used in the application. The `font-family` is set to a unique name (in this case, `__satoshi`), which can be used in the stylesheets to apply the font.

As, the users won't have the `fonts` written in the CSS file, so we use the `next/font/local` package to load the fonts. This package optimizes the loading of the fonts by generating the necessary CSS rules and making them available globally in the application.

Also, `CSS` variables are created for each font, allowing us to use them in our stylesheets like this:

```css
.font_satoshi_variable_abc123 {
  --font-satoshi: "__satoshi";
}
```

Also, `fonts` are preloaded to avoid flashing and layout shifts when the fonts are loaded. This is done by adding the `preload` attribute to the `<link>` tag in the `_document.js` file.

```bash
<link rel="preload" href="/_next/static/media/Satoshi-Regular.woff2" as="font" type="font/woff2" crossorigin="anonymous" />
```

Also, `fonts` are hashed and served from `/_next/static/media/` to ensure that the browser caches them correctly and serves the correct version of the font files.

**Build Time for Fonts**

You're **very close**, and your understanding is almost entirely correct. Let's walk through it step by step and **fine-tune** a few important points to make your mental model crystal clear.

> "After the build we will have an optimized build file..."

Yes! At **build time**, Next.js:

- Processes the font files (like `Satoshi-Regular.woff2`)
- Optimizes them (e.g., hashes file names)
- Generates `@font-face` rules
- Outputs them into the final CSS
- Generates `<link rel="preload" ...>` tags

> "...when the user requests it, the HTML gets downloaded and `<link rel="preload" ...>` runs and downloads the fonts..."

Yes! The **browser parses the HTML**, sees the `<link rel="preload" ...>` tag, and starts downloading the font **early**—before it's even needed—**to reduce render delays**.

Example:

```html
<link
  rel="preload"
  href="/_next/static/media/Satoshi-Regular.woff2"
  as="font"
  type="font/woff2"
  crossorigin="anonymous"
/>
```

This ensures **fast and smooth rendering** (no flashing text).

> "Then after download, the `@font-face` will have the link for the downloaded font and it will use it?"

🟡 **Almost right**, with a small clarification:

- The browser sees your CSS, finds this:

  ```css
  @font-face {
    font-family: "__satoshi";
    src: url("/_next/static/media/Satoshi-Regular.woff2") format("woff2");
    font-weight: 400;
    font-style: normal;
  }
  ```

- Now it knows:

  > "If something uses `font-family: __satoshi`, I should use this file at this weight/style."

- Then, it finds elements styled like:

  ```css
  html.font_satoshi_variable_xyz {
    --font-satoshi: "__satoshi";
  }

  body {
    font-family: var(--font-satoshi), sans-serif;
  }
  ```

- So the browser:

  1. Sees that `--font-satoshi` equals `__satoshi`
  2. Knows `@font-face` defines what `__satoshi` points to
  3. Uses the preloaded font file that’s already been downloaded
  4. Renders the text with `Satoshi`

**Timeline Recap:**

1. **Build time**:

   - Font files hashed and copied to `/_next/static/media/...`
   - `@font-face` CSS is generated
   - `link rel="preload"` is added to the HTML

2. **Client request**:

   - HTML is parsed
   - Fonts are downloaded early via `<link rel="preload">`
   - CSS is parsed, `@font-face` tells how to use those fonts
   - CSS variables like `--font-satoshi` are resolved
   - Text renders instantly using the font

**Note**

After the build, there will not be a single `CSS` file rather there will be multiple CSS files generated for each page and component. This is because Next.js uses **CSS Modules** and **Code Splitting** to optimize the loading of styles. Each page or component will have its own CSS file, which will be loaded only when that page or component is rendered.

Also, there will not be a single `HTML` file rather there will be multiple `HTML` files generated for each page. This is because Next.js uses **Dynamic Routing** to generate the HTML for each page based on the route. But depends on the rendering strategy used, the HTML files can be static or dynamic.

### **app**

The `app` folder will contain the main application code, including pages, components, and styles. Inside the `app` folder, we have the following structure:

```bash
app/
├── global.css
├── layout.tsx
├── page.tsx

```

The `global.css` file will contain global styles for the application. Here, we've

```Css

@theme inline {
  --font-karla: var(--font-geist-karla);
  --font-satoshi: var(--font-satoshi);
  --color-background: var(--background);
  --color-foreground: var(--foreground);
}
```

Which sets up the theme variables for fonts and colors.


Next Day understand the NextJs build process completely, we will go through the `layout.tsx` and `page.tsx` files.

**Resources**

[Link1](https://www.dhiwise.com/post/mastering-the-nextjs-build-process-for-optimal-performance)

[Link2](https://sudoshreyansh.medium.com/how-next-js-builds-your-app-6db6059ca34b)

[Link3](https://nextjs.org/docs/14/pages/building-your-application/deploying)

[Link4](https://blog.stackademic.com/understanding-the-next-js-production-build-f8d4eb23e469)

## **NextJs Build**

<hr>

When you run the `next build` command, the final build output will include the following:
